<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/5keras-multilayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilayer Feedforward Using Keras Functional API


In this exercise we train our keras model on the MNIST dataset again. But this time we use keras instead and specifically we use the keras functional API

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Flatten,Conv2D

In [2]:
def load_dataset():
  # tensorflow assumes the input and output are row vectors
  # where as in our implementation we use them as column vectors
    tr,te=keras.datasets.mnist.load_data()
    X=tr[0].astype("float32")/255
    Y=tr[1]
    #X=X.T
    Y=Y.reshape(1,len(Y))
    V=np.zeros((10,Y.shape[1]))
    for j in range(Y.shape[1]):
        V[Y[0,j],j]=1
        
    Y=V.astype("float32")
    test_data=te[0].astype("float32")/255
    test_labels=te[1].astype("float32")
    test_labels=test_labels.reshape(1,len(test_labels))
    return X,Y.T,test_data,test_labels.T


## Keras Functional API

Even though our model for this exercise is also __Sequential__ we will introduce the more flexible __functional__ API.
If we use the __Sequential__ model then our model would be
```
model= Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(10,activation="softmax"))
```

### Equivalent model using the functional API

In [3]:
input=tf.keras.layers.Input(shape=(784,))
first_dense=tf.keras.layers.Dense(128,activation="relu")(input)
second_dense=tf.keras.layers.Dense(64,activation="relu")(first_dense)
last_layer=tf.keras.layers.Dense(10,activation="softmax")(second_dense)

model=tf.keras.Model(inputs=input,outputs=last_layer)

In [4]:
X,Y,test_data,test_labels = load_dataset()

X=X.reshape(60000,784)
test_data=test_data.reshape(10000,784)


### Using the function call instead of names

__NOTE__: the labels are in one-hot encoding. If they were not then we use
```
SparseCategoricalCrossentropy instead of CategoricalCrossentropy
```


In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    #loss=keras.losses.SparseCategoricalCrossentropy(),
    loss=keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=["accuracy"]
)


### Fit model to data

In [9]:
history = model.fit(
    X,
    Y,
    batch_size=64,
    epochs=10,
   
)

Fit model on training data
Epoch 1/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0217 - accuracy: 0.9937
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0178 - accuracy: 0.9944
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0153 - accuracy: 0.9951
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0135 - accuracy: 0.9958
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0125 - accuracy: 0.9963
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0102 - accuracy: 0.9970
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0094 - accuracy: 0.9972
Epoch 8/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0086 - accuracy: 0.9973
Epoch 9/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0078 - accuracy: 0.9977
Epoch 10/10
938/938 [==============================] - 3s 3ms/step - loss

## Evaluate on test data

Note that the test_labels are **not** in one_hot encoding. We convert them to one-hot using the function
```
tf.one_hot
```

In [7]:
test_labels_one_hot=tf.one_hot(test_labels.flatten(),depth=10)
model.evaluate(test_data,test_labels_one_hot)

313/313 [==============================] - 0s 1ms/step - loss: 0.1011 - categorical_accuracy: 0.9764


[0.10110843181610107, 0.9764000177383423]

### CIFAR10

In [5]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [21]:
print(x_train.shape)
print(y_train.shape)
print(y_train[0:10])

(50000, 32, 32, 3)
(50000, 1)
[[6]
 [9]
 [9]
 [4]
 [1]
 [1]
 [2]
 [7]
 [8]
 [3]]


In [6]:
input=tf.keras.layers.Input(shape=(32,32,3))
first_layer=tf.keras.layers.Conv2D(32,(3,3),activation="relu")(input)
second_layer=tf.keras.layers.Conv2D(64,(3,3))(first_layer)
third_layer=tf.keras.layers.Flatten()(second_layer)
output=tf.keras.layers.Dense(10,activation="softmax")(third_layer)
#first_layer=Flatten()(input)
#second_layer=Dense(128,activation="relu")(first_layer)
#third_layer=Dense(64,activation="relu")(second_layer)
#output=Dense(10,activation="softmax")(third_layer)
model=tf.keras.Model(inputs=input,outputs=output)

In [7]:
model.compile(optimizer="Adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [8]:
model.fit(x_train,y_train,batch_size=128,epochs=50)

Epoch 1/50
391/391 [==============================] - 3s 7ms/step - loss: 38.0224 - accuracy: 0.3261
Epoch 2/50
391/391 [==============================] - 3s 7ms/step - loss: 2.4739 - accuracy: 0.4295
Epoch 3/50
391/391 [==============================] - 3s 7ms/step - loss: 1.6548 - accuracy: 0.5042
Epoch 4/50
391/391 [==============================] - 3s 7ms/step - loss: 1.3980 - accuracy: 0.5611
Epoch 5/50
391/391 [==============================] - 3s 7ms/step - loss: 1.2308 - accuracy: 0.6056
Epoch 6/50
391/391 [==============================] - 3s 7ms/step - loss: 1.1070 - accuracy: 0.6381
Epoch 7/50
391/391 [==============================] - 3s 7ms/step - loss: 1.0325 - accuracy: 0.6607
Epoch 8/50
391/391 [==============================] - 3s 7ms/step - loss: 0.9466 - accuracy: 0.6870
Epoch 9/50
391/391 [==============================] - 3s 7ms/step - loss: 0.8989 - accuracy: 0.7021
Epoch 10/50
391/391 [==============================] - 3s 7ms/step - loss: 0.8511 - accuracy: 0.717